<a href="https://colab.research.google.com/github/NgaAdrain/TEAM_Enter_Text_Here/blob/master/Data_Varification/20200717_Test_Model_4_csv_Modified_78_Percent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

사용에 필요한 모듈들 import

In [1]:
# Import the TensorFlow
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
tf.__version__
!pip3 install watermark

In [2]:
%load_ext watermark
%watermark -v -p numpy,scipy,sklearn,pandas,matplotlib,keras

CPython 3.6.9
IPython 5.5.0

numpy 1.18.5
scipy 1.4.1
sklearn 0.0
pandas 1.0.5
matplotlib 3.2.2
keras 2.3.1


Using TensorFlow backend.


윈도우의 크기를 32 -> 32개의 row를 하나의 window로<br>
![대충 정규화 표라는 그림](https://drive.google.com/uc?id=1VX3YCf0eue9Mx5AGc4afwm6NRseBAG0I)<br>
위의 값은 연구실에서 추출한 OBD데이터의 값의 범위표이며<br> 
이를 기반으로 시뮬레이터 데이터의 값을 정규화하여 사용할 것이다.
![대충 OBD데이터를 1차 정규화 한 값](https://drive.google.com/uc?id=1-17mEhcfdDH2zbYtvkWS5wSkCVk63W-t)<br>
실제 데이터의 예시

우리가 시뮬레이터로 부터 얻을 수 있는 값은<br>
SWA(Steering wheel), Brake, Speed, Accelerator Pos, Lateral Acc, Longitudinal Acc 의 6가지.<br>
Steering_Wheel_X, Brake, Velocity, Accel, Acceleration_Z, Accerleration_X로 mapping 가능!<br>
총 6개의 label에 대해 학습을 실시<br>
출력 라벨은 0~5<hr>

0: 직진, 1: 좌회전, 2: 우회전, 3: 좌측 차선변경, 4: 우측 차선 변경, 5: 정지.

In [13]:
WINDOW_SIZE = 32 #데이터 32개가 한 묶음(window)
train_file = pd.read_csv('/content/drive/My Drive/train_input.csv',encoding='utf-8')
train_file_label = train_file.pop('Label')
train_file.drop(['Timestamp','accel_Y','Winker(left)','Winker(right)','Timecheck'],axis = 'columns',inplace = True)
#print(train_file.values)

In [14]:
test_file = pd.read_csv('/content/drive/My Drive/test_input.csv',encoding='utf-8')
test_file_time = test_file.pop('Timestamp')
test_file_label = test_file.pop('Label')
test_file.drop(['Winker(left)','Winker(right)','Timecheck'],axis = 'columns',inplace = True)
#print(test_file.values)

데이터 정규화 과정
<br>
- 실제 데이터의 정규화내용
1. steering wheel  -> 31.75(270,-270) ~ -32
2. brake -> 31.75(50,0) ~ -32
3. speed(velocity) -> 31.75(100,0) ~ -32
4. accel -> 31.75(70,0) ~ -32
5. accelaration_longitudinal(X) -> 31.75(0.6,-0.6) ~ -32
6. accelaration_lateral(Z) -> 31.75(0.6,-0.6) ~ -32

시뮬레이터 값의 범위<br>
1. Velocity : 0 ~ 240
2. Acceleration => 분석 필요.<br>
Z -> 속도가 감소하면 양수, 속도가 증가하면 음수<br>
X -> 아직 모르겠음
3. Steering_wheel : -1 ~ 1 => -450 ~ 450(degree)
4. Brake : 0 ~ 1 => 0 ~ 100%
5. Accel : 0 ~ 1 => 0 ~ 100%<br>

=> 0716 데이터 정규화 및 학습모델 평가<br>
=> 0717 Sklearn의 Decision Tree를 구현해서 테스트
<hr>

=> 다음 주 좌측 커브, 우측 커브 추가, 급출발, 급정거 추가<br>
=> 다음 주 학습된 모델에 실제 데이터를 넣어서 테스트
 

Numpy Array로 변환.

In [15]:
train_array = np.delete(train_file.values,-1,0)
train_label = np.delete(train_file_label.values,-1,0)

In [33]:
test_array = np.delete(test_file.values,-1,0)
test_label = np.delete(test_file_label.values,-1,0)
test_time = np.delete(test_file_time.values,-1,0)

In [17]:
VELOCITY = 0
ACCEL_X = 1
ACCEL_Z = 2
STEERING = 3
ACCEL = 4
BRAKE = 5

In [35]:
for i in range(len(test_array)):
  if(test_file.values[i][VELOCITY]>=70):
    test_array[i][VELOCITY] = 31.75
  elif(test_file.values[i][VELOCITY]>=35):
    test_array[i][VELOCITY] = round(((test_array[i][VELOCITY] - 35)/35*31.75),2)
  elif(train_file.values[i][VELOCITY]>0):
    test_array[i][VELOCITY] = round(((test_array[i][VELOCITY] - 35)/35*32),2)
  else:
    test_array[i][VELOCITY] = -32
for i in range(len(test_array)):
  if(test_file.values[i][ACCEL_X]>=1):
    test_array[i][ACCEL_X] = 31.75
  elif(test_file.values[i][ACCEL_X]>= 0):
    test_array[i][ACCEL_X] = round(((test_array[i][ACCEL_X] - 0.5)/0.5*31.75),2)
  elif(test_file.values[i][ACCEL_X]> -1):
    test_array[i][ACCEL_X] = round(((test_array[i][ACCEL_X] - 0.5)/0.5*32),2)
  else:
    test_array[i][ACCEL_X] = -32
for i in range(len(test_array)):
  if(test_file.values[i][ACCEL_Z]>=1):
    test_array[i][ACCEL_Z] = 31.75
  elif(test_file.values[i][ACCEL_Z]>= 0):
    test_array[i][ACCEL_Z] = round(((test_array[i][ACCEL_Z] - 0.5)/0.5*31.75),2)
  elif(test_file.values[i][ACCEL_Z]> -1):
    test_array[i][ACCEL_Z] = round(((test_array[i][ACCEL_Z] - 0.5)/0.5*32),2)
  else:
    test_array[i][ACCEL_Z] = -32
for i in range(len(test_array)):
  if(test_file.values[i][STEERING]>=120):
    test_array[i][STEERING] = 31.75
  elif(test_file.values[i][STEERING]>= 60):
    test_array[i][STEERING] = round(((test_array[i][STEERING] - 60)/60*31.75),2)
  elif(test_file.values[i][STEERING]> 0):
    test_array[i][STEERING] = round(((test_array[i][STEERING] - 60)/60*32),2)
  else:
    test_array[i][STEERING] = -32
for i in range(len(test_array)):
  if(test_file.values[i][ACCEL]>=70):
    test_array[i][ACCEL] = 31.75
  elif(test_file.values[i][ACCEL]>= 35):
    test_array[i][ACCEL] = round(((test_array[i][ACCEL] - 35)/35*31.75),2)
  elif(test_file.values[i][ACCEL]> 0):
    test_array[i][ACCEL] = round(((test_array[i][ACCEL] - 35)/35*32),2)
  else:
    test_array[i][ACCEL] = -32
for i in range(len(test_array)):
  if(test_file.values[i][BRAKE]>=50):
    test_array[i][BRAKE] = 31.75
  elif(test_file.values[i][BRAKE]>= 25):
    test_array[i][BRAKE] = round(((test_array[i][BRAKE] - 25)/25*31.75),2)
  elif(test_file.values[i][BRAKE]> 0):
    test_array[i][BRAKE] = round(((test_array[i][BRAKE] - 25)/25*32),2)
  else:
    test_array[i][BRAKE] = -32

In [19]:
for i in range(len(train_array)):
  if(train_file.values[i][VELOCITY]>=70):
    train_array[i][VELOCITY] = 31.75
  elif(train_file.values[i][VELOCITY]>=35):
    train_array[i][VELOCITY] = round(((train_array[i][VELOCITY] - 35)/35*31.75),2)
  elif(train_file.values[i][VELOCITY]>0):
    train_array[i][VELOCITY] = round(((train_array[i][VELOCITY] - 35)/35*32),2)
  else:
    train_array[i][VELOCITY] = -32

In [20]:
for i in range(len(train_array)):
  if(train_file.values[i][ACCEL_X]>=1):
    train_array[i][ACCEL_X] = 31.75
  elif(train_file.values[i][ACCEL_X]>= 0):
    train_array[i][ACCEL_X] = round(((train_array[i][ACCEL_X] - 0.5)/0.5*31.75),2)
  elif(train_file.values[i][ACCEL_X]> -1):
    train_array[i][ACCEL_X] = round(((train_array[i][ACCEL_X] - 0.5)/0.5*32),2)
  else:
    train_array[i][ACCEL_X] = -32

In [21]:
for i in range(len(train_array)):
  if(train_file.values[i][ACCEL_Z]>=1):
    train_array[i][ACCEL_Z] = 31.75
  elif(train_file.values[i][ACCEL_Z]>= 0):
    train_array[i][ACCEL_Z] = round(((train_array[i][ACCEL_Z] - 0.5)/0.5*31.75),2)
  elif(train_file.values[i][ACCEL_Z]> -1):
    train_array[i][ACCEL_Z] = round(((train_array[i][ACCEL_Z] - 0.5)/0.5*32),2)
  else:
    train_array[i][ACCEL_Z] = -32

In [22]:
for i in range(len(train_array)):
  if(train_file.values[i][STEERING]>=120):
    train_array[i][STEERING] = 31.75
  elif(train_file.values[i][STEERING]>= 60):
    train_array[i][STEERING] = round(((train_array[i][STEERING] - 60)/60*31.75),2)
  elif(train_file.values[i][STEERING]> 0):
    train_array[i][STEERING] = round(((train_array[i][STEERING] - 60)/60*32),2)
  else:
    train_array[i][STEERING] = -32

In [23]:
for i in range(len(train_array)):
  if(train_file.values[i][ACCEL]>=70):
    train_array[i][ACCEL] = 31.75
  elif(train_file.values[i][ACCEL]>= 35):
    train_array[i][ACCEL] = round(((train_array[i][ACCEL] - 35)/35*31.75),2)
  elif(train_file.values[i][ACCEL]> 0):
    train_array[i][ACCEL] = round(((train_array[i][ACCEL] - 35)/35*32),2)
  else:
    train_array[i][ACCEL] = -32

In [24]:
for i in range(len(train_array)):
  if(train_file.values[i][BRAKE]>=50):
    train_array[i][BRAKE] = 31.75
  elif(train_file.values[i][BRAKE]>= 25):
    train_array[i][BRAKE] = round(((train_array[i][BRAKE] - 25)/25*31.75),2)
  elif(train_file.values[i][BRAKE]> 0):
    train_array[i][BRAKE] = round(((train_array[i][BRAKE] - 25)/25*32),2)
  else:
    train_array[i][BRAKE] = -32

train data를 dataset으로 만들기

In [25]:
train_dataset_data = []
train_dataset_label = []
for i in range(WINDOW_SIZE, train_array.shape[0] + 1, 1):
  train_dataset_data.append(train_array[i-WINDOW_SIZE:i])
  train_dataset_label.append(train_label[i-1])
tr_data = np.array(train_dataset_data)
tr_label = np.array(train_dataset_label) 
print(tr_data.shape)
print(tr_label.shape)

(53823, 32, 6)
(53823,)


test data를 dataset으로 만들기

In [37]:
test_dataset_data = []
test_dataset_label = []
for i in range(WINDOW_SIZE, test_array.shape[0] + 1, 1):
  test_dataset_data.append(test_array[i-WINDOW_SIZE:i])
  test_dataset_label.append(test_label[i-1])
te_data = np.array(test_dataset_data)
te_label = np.array(test_dataset_label) 
#test_array 을 te_data 대신에 넣어보자
#test_label 을 te_label 대신에 넣어보자

모델 만들기.<br>
1. column의 개수는 총 6개, 구분할 케이스의 개수도 6개.
2. window의 크기는 32. 즉 (32,6)의 데이터가 들어감.
3. Flatten을 진행
4. Dense층을 추가 -> 이번에는 3개를 지나서 최종적으로 6개를 출력하도록 해보자.
5. 첫 번째 층 -> 128
6. 두 번째 층 -> 64
7. 세 번째 층 -> 32
8. 네 번째 층 -> 16
9. 마지막 층 -> 8

In [27]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(WINDOW_SIZE, 6)),
    #keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(128,activation='relu'),
    #keras.layers.Dropout(0.2),
    keras.layers.Dense(64),
    #keras.layers.Dropout(0.2),
    keras.layers.Dense(32),
    #keras.layers.Dropout(0.2),
    keras.layers.Dense(16),
    keras.layers.Dense(8, activation='softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

모델 데이터 학습 실시.

In [ ]:
model.fit(tr_data,tr_label,epochs=100,batch_size=32)
#model.fit(tr_data,tr_label,epochs=10,batch_size=32)

모델 예측 및 학습 테스트 진행.

In [ ]:
test_loss, test_acc = model.evaluate(te_data,te_label,verbose=2)
predictions = model.predict(te_data)
count = 0
print(type(test_file_time[0]))
txt = 'Timestamp,Pred_Direction,Real_Direction\n'
for temp in te_data:
  #print(test_file_time[count],end=' ')
  txt = txt + test_file_time[count] + ' '
  if(np.argmax(predictions[count]) == 0):
  #  print('straight')
    txt = txt + ',straight'
  elif(np.argmax(predictions[count]) == 1):
  #  print('left')
    txt = txt + ',left'
  elif(np.argmax(predictions[count]) == 2):
    txt = txt + ',right'
  elif(np.argmax(predictions[count]) == 3):
    txt = txt + ',change_left' 
  elif(np.argmax(predictions[count]) == 4):
    txt = txt + ',change_right'
  else:
    txt = txt + ',stop'
  if(test_file_label[count] == 0):
    txt = txt + ',straight\n'
  elif(test_file_label[count] == 1):
    txt = txt + ',left\n'
  elif(test_file_label[count] == 2):
    txt = txt + ',right\n'
  elif(test_file_label[count] == 3):
    txt = txt + ',change_left\n'
  elif(test_file_label[count] == 4):
    txt = txt + ',change_right\n'
  else:
    txt = txt + ',stop\n'
  count = count + 1

f = open("/content/drive/My Drive/test_output.csv",'w')
f.write(txt)
f.close()


In [ ]:
test_loss, test_acc = model.evaluate(test_array, test_label, verbose=2)